In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force


In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive

/bin/sh: 1: xdg-open: not found
/bin/sh: 1: firefox: not found
/bin/sh: 1: google-chrome: not found
/bin/sh: 1: chromium-browser: not found
/bin/sh: 1: open: not found
Cannot retrieve auth tokens.
Failure("Error opening URL:https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force&state=aRAVsUwoNzR423pZKTMsY8NCBoPv4BvVCCxYTZ%2FCCLo")


In [0]:
!pwd

/content


In [0]:
import os
os.chdir("drive/SarcasmApp")

In [0]:
!pwd

/content/drive/SarcasmApp


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
!ls

dl_text  glove.6B.50d.txt  Sarcasm.ipynb  sarcasm_v2.csv
drive	 s2.py		   sarcasm.py	  sarcasm.zip


In [0]:
from dl_text import dl

Using TensorFlow backend.


In [0]:
import pandas as pd
import numpy as np
from keras.layers import Input,Dense,Dropout,merge,Conv1D,Lambda,Flatten,MaxPooling1D
from keras.models import Model
import keras
from sklearn.model_selection import train_test_split

In [0]:
def model_cnn2(dimx, dimy, embedding_matrix):
    inpx = Input(shape=(dimx,),dtype='int32',name='inpx')   
    embedx = dl.word2vec_embedding_layer(embedding_matrix)(inpx)
    inpy = Input(shape=(dimy,),dtype='int32',name='inpy')   
    embedy = dl.word2vec_embedding_layer(embedding_matrix)(inpy)
    
    sent_l = Conv1D(nb_filter=3,filter_length=2,activation='relu')(embedx)
    sent_r = Conv1D(nb_filter=3,filter_length=2,activation='relu')(embedy)
    pool_l = MaxPooling1D()(sent_l)
    pool_r = MaxPooling1D()(sent_r)

    combine  = keras.layers.Add()([pool_l,pool_r])
    flat_embed = Flatten()(combine)
    nnet_h = Dense(units=10,activation='softmax')(flat_embed)
    nnet_out = Dense(units=2,activation='softmax')(nnet_h)
    model = Model([inpx,inpy],nnet_out)
    model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
    
    return model

In [0]:
df=pd.read_csv('sarcasm_v2.csv')
df.head()

,Corpus,Label,ID,Quote Text,Response Text
0,GEN,sarc,GEN_sarc_0000,"First off, That's grade A USDA approved Libera...",Therefore you accept that the Republican party...
1,GEN,sarc,GEN_sarc_0001,watch it. Now you're using my lines. Poet has ...,More chattering from the peanut gallery? Haven...
2,GEN,sarc,GEN_sarc_0002,Because it will encourage teens to engage in r...,"Yep, suppressing natural behavior is always th..."
3,GEN,sarc,GEN_sarc_0003,Obviously you missed the point. So sorry the t...,"I guess we all missed your point Justine, what..."
4,GEN,sarc,GEN_sarc_0004,This is pure paranoia. What evidence do you ha...,"Evidence, I dont need no sticking evidence. Th..."


In [0]:
df=df.drop(columns=['Corpus','ID'])
df.head()

,Label,Quote Text,Response Text
0,sarc,"First off, That's grade A USDA approved Libera...",Therefore you accept that the Republican party...
1,sarc,watch it. Now you're using my lines. Poet has ...,More chattering from the peanut gallery? Haven...
2,sarc,Because it will encourage teens to engage in r...,"Yep, suppressing natural behavior is always th..."
3,sarc,Obviously you missed the point. So sorry the t...,"I guess we all missed your point Justine, what..."
4,sarc,This is pure paranoia. What evidence do you ha...,"Evidence, I dont need no sticking evidence. Th..."


In [0]:
df = df.replace({'sarc':1,'notsarc':0})
df.head()

,Label,Quote Text,Response Text
0,1,"First off, That's grade A USDA approved Libera...",Therefore you accept that the Republican party...
1,1,watch it. Now you're using my lines. Poet has ...,More chattering from the peanut gallery? Haven...
2,1,Because it will encourage teens to engage in r...,"Yep, suppressing natural behavior is always th..."
3,1,Obviously you missed the point. So sorry the t...,"I guess we all missed your point Justine, what..."
4,1,This is pure paranoia. What evidence do you ha...,"Evidence, I dont need no sticking evidence. Th..."


In [0]:
label = df['Label'].tolist()
quote = df['Quote Text'].tolist()
response = df['Response Text'].tolist()
print(label[0:5])
print(quote[0:5])
print(response[0:5])

[1, 1, 1, 1, 1]
["First off, That's grade A USDA approved Liberalism in a nutshell.", "watch it. Now you're using my lines. Poet has always been an easy target, I will agree. ;)", 'Because it will encourage teens to engage in riskier behavior. Abstinence until marriage is still the best way.', 'Obviously you missed the point. So sorry the the irony was beyond you.', "This is pure paranoia. What evidence do you have to support you're point of view? Why don't you actually do a little research yourself. Handgun Control Inc was founded by two victims of gun-violence. Sarah Brady joined Handgun Control Inc after her husband received brain damage from a bullet. Senator Ted Kennedy also supports gun-control because some of his family members were killed with guns. Many people who support gun control have experienced the horrors of gun violence firsthand and are just trying to make our nation a safer place. It's not because they are power hungry."]
['Therefore you accept that the Republican pa

### Cleaning Data

---



In [0]:
q=[]
r=[]
labels=[]
for sent1,sent2,l in zip(quote,response,label):
    q.append(dl.clean(sent1))
    r.append(dl.clean(sent2))
    labels.append(dl.clean(str(l)))
print(labels[0:5])
print(q[0:5])
print(r[0:5])

['1', '1', '1', '1', '1']
['First off That grade A USDA approved Liberalism in a nutshell ', 'watch it Now you are using my lines Poet has always been an easy target I will agree ; ', 'Because it will encourage teens to engage in riskier behavior Abstinence until marriage is still the best way ', 'Obviously you missed the point So sorry the the irony was beyond you ', 'This is pure paranoia What evidence do you have to support you are point of view Why do not you actually do a little research yourself Handgun Control Inc was founded by two victims of gun - violence Sarah Brady joined Handgun Control Inc after her husband received brain damage from a bullet Senator Ted Kennedy also supports gun - control because some of his family members were killed with guns Many people who support gun control have experienced the horrors of gun violence firsthand and are just trying to make our nation a safer place It not because they are power hungry ']
['Therefore you accept that the Republican par

Removing stopwords and punctuations

In [0]:
import nltk
# nltk.download()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words=set(stopwords.words('english'))

In [0]:
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

In [0]:
quotes=[]
sentence=""
for i in q:
  sentence=""
  word_tokens=word_tokenize(i)
  for w in word_tokens:
    if w not in stop_words and w not in punctuations:
      sentence+=w+' '
  quotes.append(sentence)
  del sentence

In [0]:
print(quotes[0:5])

['First That grade A USDA approved Liberalism nutshell ', 'watch Now using lines Poet always easy target I agree ', 'Because encourage teens engage riskier behavior Abstinence marriage still best way ', 'Obviously missed point So sorry irony beyond ', 'This pure paranoia What evidence support point view Why actually little research Handgun Control Inc founded two victims gun violence Sarah Brady joined Handgun Control Inc husband received brain damage bullet Senator Ted Kennedy also supports gun control family members killed guns Many people support gun control experienced horrors gun violence firsthand trying make nation safer place It power hungry ']


In [0]:
responses=[]
sentence=""
for i in r:
  sentence=""
  word_tokens=word_tokenize(i)
  for w in word_tokens:
    if w not in stop_words and w not in punctuations:
      sentence+=w+' '
  responses.append(sentence)
  del sentence

In [0]:
print(responses[0:5])

['Therefore accept Republican party almost whole grade A USDA approved Liberalism About time ', 'More chattering peanut gallery Have gotten memo longer player Honestly clamoring attention low budget No shame ', 'Yep suppressing natural behavior always way go We also get stop pooping ', 'I guess missed point Justine whatever might emoticonXConfused Better luck next time emoticonXGood ', 'Evidence I dont need sticking evidence This I beleive thats I dig proof supports point ']


In [0]:
labels=[int(i) for i in labels]
k=zip(quotes,responses)
k=list(k)
X_train,X_test,Y_train,Y_test=train_test_split(k,labels,test_size=0.20,random_state=15)
print(X_train[0:5])
print(Y_train[0:5])

[('The proof pudding I suggest read post let know disagree opinions If come conclusion God exsist God exsistance minds believers case believers different Gods throughout human history I understand referring said admit faith based preacher bad fundamentalist By way ovious I atheist ', 'Symbiote seems large hair across butt '), ('I hate break people less confidence monstosity 2000 + pages one knows could anyone even begin tell bullet points Take spam else like DU KOS received koolaid salutes ', 'An immaculately presented response entirely avoiding topic indeed reason order unwittingly case retrospective abortion '), ('I felt holy spirit take lithum kid ', 'So Christians sort brain damage trebor You getting better '), ('Afraid Well guns pretty good deterrent government trying take away rights And shooting criminals makes criminals rethink trying steal stuff ', 'Finally someone answered question Thanks I thought might government Does government attempt What think democratic nations citizen

In [0]:
quotes_train=[]
quotes_test=[]
responses_train=[]
responses_test=[]
labels_train=[]
labels_test=[]
for i in X_train:
  quotes_train.append(i[0])
  responses_train.append(i[1])
for i in X_test:
  quotes_test.append(i[0])
  responses_test.append(i[1])
for i in Y_train:
  labels_train.append(i)
for i in Y_test:
  labels_test.append(i)
print(type(labels_train))

<class 'list'>


In [0]:
labels_train = keras.utils.to_categorical(labels_train)

In [0]:
f = open('glove.6B.50d.txt', encoding="utf8")
model = {}
for line in f:
    splitline = line.split()
    word = splitline[0]
    embedding = np.array([float(val) for val in splitline[1:]])
    model[word] = embedding

In [0]:
wordvec_model = model

In [0]:
data_inp_l_train, data_inp_r_train, embedding_matrix = dl.process_data(sent_l = quotes_train, sent_r = responses_train, wordVec_model = wordvec_model, dimx = 25, dimy = 25)

found  23345  unique words
number of unkown words:  10000
some unknown words  ['$_START_$', '$_END_$', 'I', 'would', 'people']


In [0]:
model = model_cnn2(dimx = 25, dimy = 25, embedding_matrix = embedding_matrix)
model.fit([data_inp_l_train,data_inp_r_train],labels_train,epochs=10)
data_inp_l_test, data_inp_r_test, embedding_matrix = dl.process_data(sent_l = quotes_test, sent_r = responses_test, wordVec_model = wordvec_model, dimx = 25, dimy = 25)
predicted=model.predict([data_inp_l_test,data_inp_r_test])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=3, kernel_size=2)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=3, kernel_size=2)`
  


Epoch 1/10
3753/3753 [==============================] - 1s 357us/step - loss: 0.2517 - acc: 0.4999
Epoch 2/10
3753/3753 [==============================] - 1s 246us/step - loss: 0.2502 - acc: 0.4929
Epoch 3/10
3753/3753 [==============================] - 1s 265us/step - loss: 0.2501 - acc: 0.4945
Epoch 4/10
3753/3753 [==============================] - 1s 260us/step - loss: 0.2501 - acc: 0.5033
Epoch 5/10
3753/3753 [==============================] - 1s 258us/step - loss: 0.2500 - acc: 0.5001
Epoch 6/10
3753/3753 [==============================] - 1s 263us/step - loss: 0.2501 - acc: 0.4847
Epoch 7/10
3753/3753 [==============================] - 1s 259us/step - loss: 0.2501 - acc: 0.4863
Epoch 8/10
3753/3753 [==============================] - 1s 261us/step - loss: 0.2501 - acc: 0.5009
Epoch 9/10
3753/3753 [==============================] - 1s 261us/step - loss: 0.2500 - acc: 0.4924
Epoch 10/10
3104/3753 [=======================>......] - ETA: 0s - loss: 0.2500 - acc: 0.5006

3753/3753 [==============================] - 1s 259us/step - loss: 0.2501 - acc: 0.4959
found  10663  unique words
number of unkown words:  10000
some unknown words  ['$_START_$', '$_END_$', 'I', 'would', 'people']


In [0]:
correct=0
labels_test = list(keras.utils.to_categorical(labels_test))
# print(type(predicted))
k=list(predicted)
print(k[0])
print(len(labels_test[1]))
# for i in predicted:
#   if i[0]>=0.50:
#     i[0]=1
#   elif :
#     i[]
# for i in range(len(labels_test)):
#     if pred[i]==Y_test[i]:
#         correct+=1

NameError: ignored